In [60]:
import pandas as pd
import settings


In [61]:
wictionary_file_path = "kaikki.org-dictionary-English.jsonl"

with open(wictionary_file_path, "r", encoding="utf-8") as f:
    for i in range(5):
        line = f.readline().strip()

In [62]:
import json

# Define file paths
wictionary_file_path = "kaikki.org-dictionary-English.jsonl"  # TODO Replace with your actual file path
output_file = "english_translations.jsonl"

# Process the file
with open(wictionary_file_path, "r", encoding="utf-8") as f_in, open(output_file, "w", encoding="utf-8") as f_out:
    for line in f_in:
        try:
            entry = json.loads(line.strip())  # Read JSON line
            english_word = entry.get("word", "")
            pos = entry.get("pos", "NOPOS")
            raw_glosses = []
            
            if "senses" not in entry:
                continue

            for sense in entry["senses"]:
                if "raw_glosses" not in sense or "translations" not in sense:
                    continue
                
                translations = []
                for translation in sense["translations"]:
                    lang = translation.get("lang", "unknown")
                    word = translation.get("word", "")
                    translations.append({"lang": lang, "word": word})
                    raw_glosses = sense["raw_glosses"]

                if not translations:
                    continue

                json_line = json.dumps(
                    {
                        "word": english_word, 
                        "pos": pos,
                        "translations": translations,
                        "raw_glosses": raw_glosses
                    }, 
                    ensure_ascii=False)
                f_out.write(json_line + "\n")  # Write JSON line with newline
        except json.JSONDecodeError as e:
            print(f"Skipping invalid JSON line: {e}")

print(f"Success! Saved all words in {output_file}.")

Success! Saved all words in english_translations.jsonl.


In [63]:
english_translations = "english_translations.jsonl"

# Load the data
data = []
with open(english_translations, "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line.strip()))

print(f"Loaded {len(data)} entries.")

Loaded 81349 entries.


In [64]:
for entry_count, entry in enumerate(data[:5], 1):  # Print the first 5 entries
    print(f"Entry {entry_count}: {entry}")

Entry 1: {'word': 'free', 'pos': 'adj', 'translations': [{'lang': 'Afrikaans', 'word': 'vrye'}, {'lang': 'Albanian', 'word': 'lirë (i/e)'}, {'lang': 'Amharic', 'word': 'ነፃ'}, {'lang': 'Arabic', 'word': 'حُرّ'}, {'lang': 'Arabic', 'word': 'حر'}, {'lang': 'Arabic', 'word': 'حر'}, {'lang': 'Armenian', 'word': 'արձակ'}, {'lang': 'Armenian', 'word': 'ազատ'}, {'lang': 'Aromanian', 'word': 'lefter'}, {'lang': 'Assamese', 'word': 'মুকলি'}, {'lang': 'Asturian', 'word': 'llibre'}, {'lang': 'Azerbaijani', 'word': 'azad'}, {'lang': 'Bambara', 'word': 'hɔrɔn'}, {'lang': 'Bashkir', 'word': 'ирекле'}, {'lang': 'Bashkir', 'word': 'азат'}, {'lang': 'Bashkir', 'word': 'бәйһеҙ'}, {'lang': 'Bashkir', 'word': 'хөр'}, {'lang': 'Belarusian', 'word': 'свабо́дны'}, {'lang': 'Belarusian', 'word': 'во́льны'}, {'lang': 'Bengali', 'word': 'মুক্ত'}, {'lang': 'Bengali', 'word': 'আজাদ'}, {'lang': 'Bikol Central', 'word': 'talingkas'}, {'lang': 'Bulgarian', 'word': 'свобо́ден'}, {'lang': 'Catalan', 'word': 'lliure'}, 

In [ ]:
rows = []
for entry in data:
    english_word = entry["word"]
    pos = entry["pos"]
    raw_glosses = entry["raw_glosses"]
    translations = entry["translations"]

    for translation in translations:
        lang = translation["lang"]
        word = translation["word"]
        rows.append({
            "english_word": english_word,
            "pos": pos,
            "language": lang,
            "translation": word,
            "raw_glosses": raw_glosses[0]
        })


In [66]:
df = pd.DataFrame(rows)


In [74]:
pd.set_option('display.max_colwidth', None)
df.head(10)

,english_word,pos,language,translation,raw_glosses,is_long
0,free,adj,Afrikaans,vrye,"[(social) Unconstrained., Not imprisoned or enslaved.]",False
1,free,adj,Albanian,lirë (i/e),"[(social) Unconstrained., Not imprisoned or enslaved.]",False
2,free,adj,Amharic,ነፃ,"[(social) Unconstrained., Not imprisoned or enslaved.]",False
3,free,adj,Arabic,حُرّ,"[(social) Unconstrained., Not imprisoned or enslaved.]",False
4,free,adj,Arabic,حر,"[(social) Unconstrained., Not imprisoned or enslaved.]",False
5,free,adj,Arabic,حر,"[(social) Unconstrained., Not imprisoned or enslaved.]",False
6,free,adj,Armenian,արձակ,"[(social) Unconstrained., Not imprisoned or enslaved.]",False
7,free,adj,Armenian,ազատ,"[(social) Unconstrained., Not imprisoned or enslaved.]",False
8,free,adj,Aromanian,lefter,"[(social) Unconstrained., Not imprisoned or enslaved.]",False
9,free,adj,Assamese,মুকলি,"[(social) Unconstrained., Not imprisoned or enslaved.]",False


In [68]:
# Anzahl der Einträge pro Sprache
print(df["language"].value_counts())

# Anzahl der Einträge pro englischem Wort
print(df["english_word"].value_counts())

language
Finnish      70359
German       61777
Russian      54408
Spanish      51622
French       49372
             ...  
Dongolawi        1
Dompo            1
Domari           1
Dobu             1
Magyar           1
Name: count, Length: 4006, dtype: int64
english_word
water           4025
dog             1060
rain             975
order            927
fish             896
                ... 
neologize          1
oblique            1
cytomegaly         1
topspace           1
partitionist       1
Name: count, Length: 57231, dtype: int64


In [69]:
lang_long = df['language'].unique()
lang_long = pd.DataFrame(lang_long, columns=['language'])
lang_long

,language
0,Afrikaans
1,Albanian
2,Amharic
3,Arabic
4,Armenian
...,...
4001,Church Slavonic
4002,Shaozhou Tuhua
4003,Finish
4004,Ulster Scots


In [70]:
lang_long = lang_long[lang_long['language'].str.contains(',') | lang_long['language'].str.contains(r'\d')]

lang_long

for lang in lang_long["language"]:
    print(lang)

1895, Sir Arthur Conan Doyle, chapter 12, in The Stark Munro Letters


In [71]:
df['is_long'] = df["language"].apply(lambda x: x in list(lang_long["language"]))
cleaned = df[df["is_long"] == False]

print(df.shape[0], cleaned.shape[0])


1197714 1197713


In [72]:
cleaned.head(100)

,english_word,pos,language,translation,raw_glosses,is_long
0,free,adj,Afrikaans,vrye,"[(social) Unconstrained., Not imprisoned or en...",False
1,free,adj,Albanian,lirë (i/e),"[(social) Unconstrained., Not imprisoned or en...",False
2,free,adj,Amharic,ነፃ,"[(social) Unconstrained., Not imprisoned or en...",False
3,free,adj,Arabic,حُرّ,"[(social) Unconstrained., Not imprisoned or en...",False
4,free,adj,Arabic,حر,"[(social) Unconstrained., Not imprisoned or en...",False
...,...,...,...,...,...,...
95,free,adj,Polish,swobodny,"[(social) Unconstrained., Not imprisoned or en...",False
96,free,adj,Portuguese,livre,"[(social) Unconstrained., Not imprisoned or en...",False
97,free,adj,Quechua,qispi,"[(social) Unconstrained., Not imprisoned or en...",False
98,free,adj,Romagnol,lèbar,"[(social) Unconstrained., Not imprisoned or en...",False


In [73]:
cleaned["word_and_gloss"] = cleaned["english_word"] + settings.SEPERATOR + cleaned["raw_glosses"]
cleaned["language_and_translation"] = cleaned["language"] + settings.SEPERATOR + cleaned["translation"] + settings.SEPERATOR + cleaned["pos"]
cleaned.head()

TypeError: can only concatenate str (not "list") to str

In [ ]:
cleaned = cleaned.drop(columns=["english_word","translation", "is_long", "raw_glosses", "language", "pos"])
cleaned = cleaned.drop_duplicates()
cleaned

,word_and_gloss,language_and_translation
0,free - (social) Unconstrained.; Not imprisoned...,Afrikaans: vrye: adj
1,free - (social) Unconstrained.; Not imprisoned...,Albanian: lirë (i/e): adj
2,free - (social) Unconstrained.; Not imprisoned...,Amharic: ነፃ: adj
3,free - (social) Unconstrained.; Not imprisoned...,Arabic: حُرّ: adj
4,free - (social) Unconstrained.; Not imprisoned...,Arabic: حر: adj
...,...,...
1197709,fawn response - (psychology) An overadaptation...,Finnish: miellyttämisreaktio: noun
1197710,fawn response - (psychology) An overadaptation...,German: Bambi-Reflex: noun
1197711,fawn response - (psychology) An overadaptation...,German: Unterwerfungsreaktion: noun
1197712,"one heart, one soul - (Philippines, Catholicis...",Latin: cor ūnum et anima ūna: phrase


In [ ]:
cleaned.to_parquet("cleaned_dataframe.parquet", engine="pyarrow")

In [ ]:
len(cleaned['word_and_gloss'].unique())

81346